In [1]:
import os
import pickle
import pandas as pd
import numpy as np

from src.io import read_pkl
from src.default_paths import path_root
from src.mappings import model_names, task_names

In [2]:
PATH_TO_ADAPTER_MODELS_RESULTS = os.path.join(path_root, "data", "evaluate", "adapter_models")
PATH_TO_CLMBR_TASK_RESULTS = os.path.join(path_root, "data", "evaluate", "clmbr_finetuned")

In [3]:
def list_dir(path: str):
    """get list of file/directory names excluding nb checkpoints"""
    
    return [
        x for x in os.listdir(path)
        if x != ".ipynb_checkpoints"
    ]

def get_bstrp_results(
    results: dict, 
    metric: str = "auroc", 
    rounding: int = 3,
    formatted_string: bool = True
    ):
    """return formatted string of bootstrap results"""
    
    lower, upper = (
        np.percentile(results[f"{metric}_bootstrap"],[2.5, 97.5])
        .round(rounding)
    )
    
    pe = results[metric].round(rounding)
    
    if formatted_string:
        return f"{str(pe)} [{str(lower)}, {str(upper)}]"
    else:
        return (lower, pe, upper)

In [4]:
use_boot = True

auroc_results = {}
auprc_results = {}

models = list_dir(PATH_TO_ADAPTER_MODELS_RESULTS)

for model in models:
    tasks = list_dir(os.path.join(PATH_TO_ADAPTER_MODELS_RESULTS, model))
    auroc_results[model] = {}
    auprc_results[model] = {}
    
    for task in tasks:
        try:
            result = read_pkl(os.path.join(PATH_TO_ADAPTER_MODELS_RESULTS, model, task, "results.pkl"))
            auroc_results[model][task] = get_bstrp_results(result, "auroc") if use_boot else result["auroc"]
            auprc_results[model][task] = get_bstrp_results(result, "auprc") if use_boot else result["auprc"]
        except:
            auroc_results[model][task] = np.nan
            auprc_results[model][task] = np.nan

models = list_dir(PATH_TO_CLMBR_TASK_RESULTS)

for model in models:
    tasks = list_dir(os.path.join(PATH_TO_CLMBR_TASK_RESULTS, model))
    auroc_results[model+"_ft"] = {}
    auprc_results[model+"_ft"] = {}
    
    for task in tasks:
        try:
            result = read_pkl(os.path.join(PATH_TO_CLMBR_TASK_RESULTS, model, task, "results.pkl"))
            auroc_results[model+"_ft"][task] =  get_bstrp_results(result, "auroc") if use_boot else result["auroc"]
            auprc_results[model+"_ft"][task] = get_bstrp_results(result, "auprc") if use_boot else result["auprc"]
        except:
            auroc_results[model][task] = np.nan
            auprc_results[model][task] = np.nan

In [5]:
column_orders = [
    "GBM", 
    "GBM [No Expansion]", 
    "CLMBR_SK",
    "MOTOR_SK",
    "CLMBR", 
    "CLMBR_PAPT", 
    "CLMBR_CAPT",
]

task_orders = [
    "In-hospital Mortality", "Long LOS", "30-day Readmission", 
    #"In-hospital Mortality", "Long LOS", "30-day Readmission", 
    "Hypoglycemia", "Hyperkalemia", "Hyponatremia",
    "Thrombocytopenia", "Anemia"
]

In [6]:
df_auroc = (
    pd.DataFrame.from_dict(auroc_results)
    .round(4)
    .rename(columns=model_names)
    .reset_index(names="Task")
    .replace(task_names)
    .set_index("Task")
    .reindex(task_orders)
    [column_orders]
)

df_auroc

,GBM,GBM [No Expansion],CLMBR_SK,MOTOR_SK,CLMBR,CLMBR_PAPT,CLMBR_CAPT
Task,,,,,,,
In-hospital Mortality,"0.934 [0.9, 0.964]","0.893 [0.815, 0.953]","0.968 [0.952, 0.981]","0.954 [0.927, 0.975]","0.941 [0.902, 0.971]","0.957 [0.922, 0.98]","0.963 [0.937, 0.982]"
Long LOS,"0.874 [0.862, 0.886]","0.866 [0.853, 0.879]","0.847 [0.834, 0.861]","0.828 [0.813, 0.842]","0.815 [0.8, 0.83]","0.839 [0.825, 0.853]","0.836 [0.822, 0.85]"
30-day Readmission,"0.801 [0.775, 0.825]","0.783 [0.755, 0.809]","0.774 [0.746, 0.799]","0.821 [0.797, 0.844]","0.774 [0.747, 0.799]","0.804 [0.78, 0.828]","0.808 [0.783, 0.832]"
Hypoglycemia,"0.882 [0.831, 0.926]","0.88 [0.83, 0.924]","0.924 [0.891, 0.953]","0.887 [0.841, 0.927]","0.915 [0.879, 0.945]","0.918 [0.883, 0.948]","0.915 [0.879, 0.947]"
Hyperkalemia,"0.762 [0.7, 0.82]","0.749 [0.687, 0.808]","0.848 [0.812, 0.883]","0.807 [0.758, 0.853]","0.793 [0.743, 0.838]","0.822 [0.781, 0.86]","0.818 [0.778, 0.856]"
Hyponatremia,"0.815 [0.644, 0.945]","0.783 [0.579, 0.957]","0.936 [0.882, 0.977]","0.908 [0.801, 0.976]","0.925 [0.88, 0.963]","0.957 [0.933, 0.978]","0.952 [0.929, 0.974]"
Thrombocytopenia,"0.951 [0.922, 0.974]","0.953 [0.928, 0.975]","0.966 [0.953, 0.978]","0.948 [0.924, 0.969]","0.962 [0.946, 0.975]","0.967 [0.95, 0.98]","0.967 [0.954, 0.979]"
Anemia,"0.918 [0.893, 0.94]","0.919 [0.898, 0.938]","0.932 [0.912, 0.95]","0.924 [0.904, 0.941]","0.918 [0.897, 0.937]","0.946 [0.932, 0.959]","0.941 [0.927, 0.954]"


In [7]:
df_auprc = (
    pd.DataFrame.from_dict(auprc_results)
    .round(3)
    .rename(columns=model_names)
    .reset_index(names="Task")
    .replace(task_names)
    .set_index("Task")
    .reindex(task_orders)
    [column_orders]
)

df_auprc

,GBM,GBM [No Expansion],CLMBR_SK,MOTOR_SK,CLMBR,CLMBR_PAPT,CLMBR_CAPT
Task,,,,,,,
In-hospital Mortality,"0.325 [0.186, 0.489]","0.346 [0.202, 0.513]","0.273 [0.164, 0.44]","0.181 [0.108, 0.312]","0.343 [0.2, 0.492]","0.386 [0.236, 0.544]","0.422 [0.271, 0.568]"
Long LOS,"0.62 [0.586, 0.653]","0.612 [0.58, 0.645]","0.556 [0.52, 0.589]","0.525 [0.491, 0.56]","0.49 [0.456, 0.526]","0.534 [0.499, 0.57]","0.531 [0.497, 0.567]"
30-day Readmission,"0.32 [0.271, 0.373]","0.323 [0.273, 0.375]","0.302 [0.255, 0.353]","0.357 [0.304, 0.411]","0.265 [0.222, 0.314]","0.307 [0.26, 0.359]","0.303 [0.258, 0.357]"
Hypoglycemia,"0.202 [0.136, 0.306]","0.202 [0.129, 0.3]","0.303 [0.205, 0.416]","0.176 [0.118, 0.269]","0.218 [0.146, 0.321]","0.225 [0.157, 0.329]","0.217 [0.151, 0.32]"
Hyperkalemia,"0.071 [0.044, 0.128]","0.054 [0.033, 0.096]","0.097 [0.06, 0.167]","0.084 [0.052, 0.145]","0.061 [0.037, 0.113]","0.07 [0.045, 0.12]","0.065 [0.042, 0.112]"
Hyponatremia,"0.038 [0.008, 0.182]","0.034 [0.009, 0.137]","0.05 [0.02, 0.117]","0.112 [0.025, 0.331]","0.041 [0.012, 0.159]","0.047 [0.02, 0.114]","0.043 [0.017, 0.128]"
Thrombocytopenia,"0.68 [0.594, 0.759]","0.689 [0.604, 0.769]","0.527 [0.43, 0.628]","0.478 [0.383, 0.581]","0.486 [0.391, 0.588]","0.531 [0.438, 0.628]","0.514 [0.419, 0.619]"
Anemia,"0.401 [0.321, 0.485]","0.413 [0.333, 0.494]","0.388 [0.311, 0.482]","0.348 [0.273, 0.431]","0.338 [0.265, 0.419]","0.424 [0.342, 0.511]","0.416 [0.334, 0.501]"


In [8]:
if use_boot:
    df_auroc.to_csv("../results/main_auroc_boot.csv")
    df_auprc.to_csv("../results/main_auprc_boot.csv")
else:
    df_auroc.to_csv("../results/main_auroc.csv")
    df_auprc.to_csv("../results/main_auprc.csv")